In [1]:
import pandas as pd
audio_impro_processed = pd.read_csv('audio_df_improvised.csv')

audio_impro_processed.reset_index(inplace=True)
audio_impro_processed.head()

,index,start_time,end_time,wav_file,emotion,val,act,dom
0,0,6.2901,8.2357,Ses01F_impro01_F000,neu,2.5,2.5,2.5
1,1,10.0100,11.3925,Ses01F_impro01_F001,neu,2.5,2.5,2.5
2,2,14.8872,18.0175,Ses01F_impro01_F002,neu,2.5,2.5,2.5
3,3,27.4600,31.4900,Ses01F_impro01_F005,neu,2.5,3.5,2.0
4,4,85.2700,88.0200,Ses01F_impro01_F012,ang,2.0,3.5,3.5


In [114]:
from torchvision import models, transforms
my_transforms=transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [129]:
import librosa
from PIL import Image
def get_spectrogram_image(path, my_transforms, med_duration, sr=None, n_fft=2048, hop_length=256, n_mels=128, fmin=100, fmax=15000, top_db=80):
    med_duration = int(np.floor(med_duration))
    wav, sr = librosa.load(path)
    if wav.shape[0]<(med_duration*sr):
        wav=np.pad(wav,int(np.ceil(((med_duration*sr)-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:(med_duration*sr)]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels)
#     librosa.decompose.nn_filter(spec, aggregate=np.median, metric='cosine')
    spec_db=librosa.power_to_db(spec)
    img=np.stack((spec_db,)*3, axis=-1)
    PIL_image = Image.fromarray((img*255).astype(np.uint8))
    image=my_transforms(PIL_image)
    return PIL_image

In [140]:
import matplotlib.pyplot as plt

a = get_spectrogram_image('/data/home/advaitmb/datasets/sentences/' + audio_impro_processed['wav_file'][1] + '.wav', my_transforms, 3)

from IPython.display import Image as Im

display(Im(a))

TypeError: 'module' object is not callable

The Following code validates on the Male speaker of last Session 

In [116]:
# We split the dataframe into train and test segments

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

train, valid = train_test_split(audio_impro_processed, test_size=0.2, shuffle=False)
train = audio_impro_processed[:2617]
train = shuffle(train)
valid = audio_impro_processed[267:]

In [117]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os

class IEMOCAP_3Channel(Dataset):
    def __init__(self, base, df, in_col, out_col):
        self.df = df
        self.data = []
        self.labels = []
        self.c2i={}
        self.i2c={}
        self.categories = sorted(df[out_col].unique())
        print(self.categories)
        for i, category in enumerate(self.categories):
            self.c2i[category]=i
            self.i2c[i]=category
        for ind in tqdm(range(len(df))):
            row = df.iloc[ind]
            
            #If all files are stored in a folder named sentences
            file_path = base + '/' + row[in_col] + '.wav'
            
            
            self.data.append(get_spectrogram_image(file_path, my_transforms, 4))
            self.labels.append(self.c2i[row['emotion']])
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]





In [118]:
train_data = IEMOCAP_3Channel('/data/home/advaitmb/datasets/sentences', train, 'wav_file', 'emotion')
valid_data = IEMOCAP_3Channel('/data/home/advaitmb/datasets/sentences', valid, 'wav_file', 'emotion')
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=16)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True, num_workers=16)

  0%|          | 1/2617 [00:00<05:14,  8.31it/s]

['ang', 'hap', 'neu', 'sad']


  0%|          | 1/2676 [00:00<05:56,  7.50it/s]

['ang', 'hap', 'neu', 'sad']


 39%|███▉      | 1049/2676 [02:28<03:50,  7.04it/s]


KeyboardInterrupt: 

In [64]:
for i, data in enumerate(train_loader):
    print(data[0].shape)

torch.Size([5, 3, 224, 224])


In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F

net = models.resnet18(pretrained=True)
net.fc = nn.Linear(512, 4)

In [94]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')

net = net.to(device)

In [95]:
def setlr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

def lr_decay(optimizer, epoch):
    if epoch%10==0:
        new_lr = learning_rate / (10**(epoch//10))
        optimizer = setlr(optimizer, new_lr)
        print("Changed learning rate to {}".format(new_lr))
    return optimizer

In [96]:
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.01)
epochs = 20
loss_fn = nn.CrossEntropyLoss()
resnet_train_losses=[]
resnet_valid_losses=[]

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score


def train_model(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
    
    for epoch in tqdm(range(1,epochs+1)):
        
        model.train()
        batch_losses=[]
        if change_lr:
            optimizer = change_lr(optimizer, epoch)
        for i, data in enumerate(train_loader):
            x, y = data
            
            optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        train_losses.append(batch_losses)


        print("Epoch - {} Train-Loss : {}".format(epoch, np.mean(train_losses[-1])))
        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        for i, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())

        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
#         accuracy_unweighted = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        accuracy_unweighted = accuracy_score(trace_yhat.argmax(axis=1), trace_y)
        accuracy_weighted = balanced_accuracy_score(trace_yhat.argmax(axis=1), trace_y)
        print("Epoch - {} Valid-Loss : {} Valid-Accuracy Unweighted : {} Valid-Accuracy Weighted {}".format(epoch, np.mean(valid_losses[-1]), accuracy_unweighted, accuracy_weighted ))
        



In [97]:
train_model(net, loss_fn, train_loader, valid_loader, epochs, optimizer, resnet_train_losses, resnet_valid_losses, lr_decay)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch - 1 Train-Loss : 1.1526133265437148


  5%|▌         | 1/20 [00:21<06:49, 21.54s/it]

Epoch - 1 Valid-Loss : 1.1260659803043713 Valid-Accuracy Unweighted : 0.49079754601226994 Valid-Accuracy Weighted 0.3808571364325547
Epoch - 2 Train-Loss : 0.8994067747418474


 10%|█         | 2/20 [00:43<06:27, 21.55s/it]

...

Epoch - 18 Valid-Loss : 1.398373159495267 Valid-Accuracy Unweighted : 0.4815950920245399 Valid-Accuracy Weighted 0.3881888440860215
Epoch - 19 Train-Loss : 0.053146671276630426


 95%|█████████▌| 19/20 [06:55<00:21, 21.94s/it]

Epoch - 19 Valid-Loss : 1.4643620415167375 Valid-Accuracy Unweighted : 0.46319018404907975 Valid-Accuracy Weighted 0.38102583020173975
Changed learning rate to 2.0000000000000002e-07
Epoch - 20 Train-Loss : 0.050683296898879654


100%|██████████| 20/20 [07:17<00:00, 21.86s/it]

Epoch - 20 Valid-Loss : 1.3873401338403875 Valid-Accuracy Unweighted : 0.4785276073619632 Valid-Accuracy Weighted 0.3872326524643962
